In [12]:
import tensorflow as tf
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Conv2D, Flatten, MaxPooling2D, Input
import matplotlib.pyplot as plt
from keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
import numpy as np
import time
import keras
from keras.models import Sequential

In [2]:
#X = np.load('features_1000_32.npy')/255
#y = np.load('labels_1000_32.npy')
#X = np.load("aug_features_32x32.npy")/255
#y = np.load("aug_labels_32x32.npy")

#X = np.load('features_std.npy')/255
#y = np.load('labels_std.npy')

#X = np.load('features_png.npy')
#y = np.load('labels_png.npy')


In [ ]:
X = np.load('features_std.npy')/255
y = np.load('labels_std.npy')

X_test = np.load('test_features.npy')/255
y_test = np.load('test_labels.npy')

In [ ]:
# concatenate train and test set

X = np.concatenate((X,X_test), axis=0)
print(X.shape)
y = np.concatenate((y,y_test), axis=0)
print(y.shape)

In [4]:
# first split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=22)


In [5]:
# split test set into real test set and small validation set
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.3, random_state=10)

In [6]:
NAME = "CNN_16_64_128_{}".format(int(time.time()))

tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))

In [10]:
inputs = Input(shape=(X.shape[1],X.shape[2], 3))

x = Conv2D(16, (3,3), padding="same", activation='relu')(inputs)
#x = MaxPooling2D(pool_size=(2,2))(x)
block_1_output = Dropout(0)(x)


x = Conv2D(64, (3,3), padding="same", activation='relu')(block_1_output)
x = MaxPooling2D(pool_size=(2,2))(x)
block_2_output = Dropout(0)(x)

x = Conv2D(128, (3,3), padding="same", activation='relu')(block_2_output)
x = MaxPooling2D(pool_size=(2,2))(x)
block_3_output = Dropout(0)(x)

#x = Conv2D(128, (3,3), padding="same", activation='relu')(block_3_output)
#x = MaxPooling2D(pool_size=(2,2))(x)
#block_4_output = Dropout(0)(x)

x = Flatten()(block_3_output)
x = Dense(256, activation='relu')(x)

prediction = Dense(y.shape[1], activation='softmax')(x)
#prediction = Dense(17, activation='softmax')(x)
model_1 = Model(inputs=inputs, outputs=prediction)


In [11]:
model_1.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])

In [9]:
model_1.fit(X_train, y_train, batch_size=64, epochs=10, validation_data=(X_val,y_val), callbacks=[tensorboard])

Train on 38879 samples, validate on 3888 samples
Epoch 1/30
  448/38879 [..............................] - ETA: 4:22 - loss: 3.6309 - accuracy: 0.0647

/home/darius/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.103510). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


38879/38879 [==============================] - 22s 568us/step - loss: 0.4977 - accuracy: 0.8648 - val_loss: 0.0642 - val_accuracy: 0.9833
Epoch 2/30
38879/38879 [==============================] - 19s 489us/step - loss: 0.0508 - accuracy: 0.9868 - val_loss: 0.0551 - val_accuracy: 0.9871
Epoch 3/30
 1952/38879 [>.............................] - ETA: 17s - loss: 0.0248 - accuracy: 0.9913

KeyboardInterrupt: 

In [7]:
model_1.save('CNN_16_64_128_std_train')
